In [254]:
import IPython.display as ipd
import matplotlib.pyplot as plt
import librosa.display
import time
import warnings; warnings.simplefilter('ignore')
import pandas as pd
import numpy as np
import youtube_dl
import librosa
import os
import re
import tensorflow as tf

from tensorflow.keras import optimizers
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Sequential
from __future__ import unicode_literals
from os import path, listdir 
from os.path import isfile, join
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [255]:
format_music  = '.wav'

In [256]:
def two_d_to_one(data):
    data['Chroma_stft_f'] = data['Chroma_stft'].apply(lambda x: x[0])
    data['Chroma_stft_s'] = data['Chroma_stft'].apply(lambda x: x[1])

    data['Flatness'] = data['Flatness'].apply(lambda x: x[0])

    #data['Tempogram_f'] = data['Tempogram'].apply(lambda x: x[0])
    #data['Tempogram_s'] = data['Tempogram'].apply(lambda x: x[1])

    data = data.drop(columns="Chroma_stft")
    data = data.drop(columns="Flatness")
    data = data.drop(columns="Tempogram")
    return data

In [257]:
def extract_features(data, mono):
    if mono:
        data['Chroma_stft'] = data['Librosa_song'].apply(lambda y : librosa.feature.chroma_stft(y = y, sr = 44100))
        data['Flatness'] = data['Librosa_song'].apply(lambda y :librosa.feature.spectral_flatness(y = y))
        data['Tonnetz'] = data['Librosa_song'].apply(lambda y :librosa.effects.harmonic(y = y))
        data['Tempogram'] = data['Librosa_song'].apply(lambda y:
            librosa.feature.tempogram(
                onset_envelope=librosa.onset.onset_strength(y = y, sr = 44100), 
                sr = 44100)
        )
        return data
    else:
        data['Chroma_stft_x'] = data['Librosa_song'].apply(lambda y : librosa.feature.chroma_stft(y = y[0], sr = 44100))
        data['Chroma_stft_y'] = data['Librosa_song'].apply(lambda y : librosa.feature.chroma_stft(y = y[1], sr = 44100))
        
        data['Flatness_x'] = data['Librosa_song'].apply(lambda y :librosa.feature.spectral_flatness(y = y[0]))
        data['Flatness_y'] = data['Librosa_song'].apply(lambda y :librosa.feature.spectral_flatness(y = y[1]))
        
        data['Tonnetz'] = data['Librosa_song'].apply(lambda y :np.array([librosa.effects.harmonic(y = y[0]),
                                                                           librosa.effects.harmonic(y = y[1])]
                                                                         ))
        data['Tempogram_x'] = data['Librosa_song'].apply(lambda y:
            librosa.feature.tempogram(
                onset_envelope=librosa.onset.onset_strength(y = y[0], sr = 44100), 
                sr = 44100)
        )
        data['Tempogram_y'] = data['Librosa_song'].apply(lambda y:
            librosa.feature.tempogram(
                onset_envelope=librosa.onset.onset_strength(y = y[1], sr = 44100), 
                sr = 44100)
        )
        return data

In [258]:
def create_music_album(mono):
    music_album = {}
    directories = [i for i in listdir("music/") if not i.startswith(".")]
    for songs_dir in directories:
        music_album[songs_dir] = []
        for song in listdir(f"music/{songs_dir}"):
            add = True
            if not song.startswith(".") and not song == 'song_list':
                audio_data = f'music/{songs_dir}/{song}'
                x , sr = librosa.load(audio_data, mono=mono, sr=44100)    #Cambiar a x , sr = librosa.load(audio_data, mono=False) para ver como afecta
                if mono:
                    part_song = divide_by_sec(x, sr)
                else:
                    if len(x) == 2:
                        p1 = divide_by_sec(x[0], sr)
                        p2 = divide_by_sec(x[1], sr)
                        part_song = np.array(list(zip(p1,p2)))
                    else:
                        add = False
                if add:
                    music_album[songs_dir].append(part_song)
    return music_album

In [259]:
def dataFrame_music(mono):
    music_album = create_music_album(mono)
    lista_songs = []
    for genero in music_album:
        for song in range(len(music_album[genero])):
            for part_songs in music_album[genero][song]:
                lista_songs.append([part_songs, genero])
            
    dfsongs = pd.DataFrame(np.array(lista_songs), columns=['Librosa_song', 'Genre'])
    dfsongs = extract_features(dfsongs, mono)
    if mono:
        dfsongs = two_d_to_one(dfsongs)
    return dfsongs

In [260]:
def clean_name(song_name):
    to_clean = [" (Audio Oficial)", " (Videoclip Oficial)", " , letra", " Letra", " (oficial audio)", " (Audio)", " LETRA", " (Letra)", " (Lyric Video)", " (Lyric Video Oficial)", ]
    for delete_part in to_clean:
        song_name = song_name.replace(delete_part, "")
    return song_name

In [261]:
def clean_all_directories():
    directories = [i for i in listdir("music/") if not i.startswith(".")]
    for type_music in directories:
        for song in listdir(f"music/{type_music}"):
            if song != "song_list" and not song.startswith("."):
                os.remove(f"music/{type_music}/{song}")

In [262]:
def divide_by_sec(audio, sr, seconds = 5):
    divided_audio = []
    for it in range(1, round(len(audio)//(seconds*sr))):
        start = (it-1)*seconds*sr
        end = it*seconds*sr
        divided_audio.append(audio[start:end])
    return divided_audio

In [90]:
7.9//2

3.0

In [ ]:
music_album = {}
directories = [i for i in listdir("music/") if not i.startswith(".")]
for songs_dir in directories:
    music_album[songs_dir] = []
    for song in listdir(f"music/{songs_dir}"):
        add = True
        if not song.startswith(".") and not song == 'song_list':
            audio_data = f'music/{songs_dir}/{song}'
            x , sr = librosa.load(audio_data, mono=mono, sr=44100)    #Cambiar a x , sr = librosa.load(audio_data, mono=False) para ver como afecta
            if mono:
                part_song = divide_by_sec(x, sr)
            else:
                if len(x) == 2:
                    p1 = divide_by_sec(x[0], sr)
                    p2 = divide_by_sec(x[1], sr)
                    part_song = np.array(list(zip(p1,p2)))
                else:
                    add = False
            if add:
                music_album[songs_dir].append(part_song)

lista_songs = []
for genero in music_album:
    for song in range(len(music_album[genero])):
        for part_songs in music_album[genero][song]:
            lista_songs.append([part_songs, genero])

dfsongs = pd.DataFrame(np.array(lista_songs), columns=['Librosa_song', 'Genre'])
dfsongs = extract_features(dfsongs, mono)
if mono:
    dfsongs = two_d_to_one(dfsongs)

In [263]:
%%time
one_dim = dataFrame_music(True)
one_dim

CPU times: user 4min 26s, sys: 4min 13s, total: 8min 40s
Wall time: 3min 24s


,Librosa_song,Genre,Tonnetz,Chroma_stft_f,Chroma_stft_s
0,"[-0.026925595, -0.048504688, -0.06306862, -0.0...",classical,"[-0.040174346, -0.04306239, -0.046940167, -0.0...","[0.36120847, 0.38205665, 0.28405944, 0.4626564...","[0.28653854, 0.2143556, 0.18710317, 0.2855665,..."
1,"[0.0002749279, -0.0011622504, -0.0017395996, -...",classical,"[-0.00088757323, -0.00082119333, -0.0003863558...","[1.0, 1.0, 1.0, 0.7593036, 0.44775972, 0.35638...","[0.24926227, 0.44633397, 0.7172283, 0.17598368..."
2,"[-0.11859189, -0.11516097, -0.11441008, -0.118...",classical,"[-0.1024951, -0.101341985, -0.09922943, -0.098...","[1.0, 0.64629424, 0.49779817, 0.5285119, 0.514...","[0.43126613, 0.1850466, 0.19316506, 0.24085873..."
3,"[-0.00033854484, -0.003145342, -0.0061922697, ...",classical,"[-0.0024653536, -0.003110117, -0.0045063505, -...","[0.14908326, 0.16485389, 0.21698251, 0.1323248...","[0.112471215, 0.09150309, 0.23798779, 0.226464..."
4,"[-0.0048522064, -0.0036253056, -0.0027464728, ...",classical,"[-0.0051691798, -0.0049385526, -0.0045592985, ...","[1.0, 1.0, 0.63672036, 0.43627167, 0.55705327,...","[0.55648285, 0.42119658, 0.2512289, 0.12701692..."
...,...,...,...,...,...
292,"[0.118544236, 0.16745976, 0.19948018, 0.236261...",country,"[0.078795135, 0.08875732, 0.11199952, 0.141054...","[0.24605285, 0.21903959, 0.24440855, 0.3291759...","[0.17209417, 0.089031376, 0.059214808, 0.08174..."
293,"[0.18821052, 0.18545504, 0.1821335, 0.17911138...",country,"[0.16581914, 0.16436425, 0.16121632, 0.1580679...","[0.097745694, 0.13378851, 0.1511519, 0.096007,...","[0.101187654, 0.0954254, 0.09675367, 0.0983222..."
294,"[0.24029963, 0.23842339, 0.2324136, 0.224482, ...",country,"[0.20979278, 0.20866165, 0.2052093, 0.20114154...","[0.021224316, 0.021564202, 0.017289894, 0.0159...","[0.03054991, 0.029357428, 0.029947417, 0.03079..."
295,"[0.42963752, 0.3886683, 0.3469949, 0.30317912,...",country,"[0.1976423, 0.1914004, 0.17701831, 0.1621761, ...","[0.10485836, 0.12167017, 0.10300083, 0.1053118...","[0.07270728, 0.095013015, 0.1177889, 0.1166963..."


In [ ]:
%%time
two_dim = dataFrame_music(False)
two_dim

In [ ]:
#clean_all_directories()

In [264]:
X_dummies = pd.get_dummies(one_dim, columns=['Genre'])
y = X_dummies[re.findall(r"Genre\w*", " ".join(X_dummies.columns))]
X_dummies = X_dummies.drop(columns=re.findall(r"Genre\w*", ",".join(X_dummies.columns)))

In [50]:
test_unified = unified_data(X_dummies)
test_unified

NameError: name 'unified_data' is not defined

In [ ]:
test_unified.shape

In [265]:
X_train, X_val, y_train, y_val = train_test_split(X_dummies, y, train_size=0.5, random_state=42)

In [266]:
def unified_data(dataFrame):
    index = []
    data = []
    for rows in dataFrame.iterrows():
        index.append(rows[0])
        unified_colums = []
        for column_data in rows[1].tolist():
            unified_colums.extend(column_data)
        data.append(unified_colums)
    return np.array(data)

In [267]:
def unified_result(dataFrame):
    index = []
    data = []
    for rows in dataFrame.iterrows():
        unified_colums = []
        for column_data in rows[1].tolist():
            unified_colums.append(column_data)
        data.append(np.array(unified_colums))
    return np.array(data)

In [268]:
X_train_unified = unified_data(X_train)
X_val_unified = unified_data(X_val)
y_train_unified = unified_result(y_train)
y_val_unified = unified_result(y_val)

In [58]:
i=1

In [63]:
X_train.iloc[i,2]

array([0.20356572, 0.08098475, 0.04160352, 0.03815017, 0.02164948,
       0.02281864, 0.03007324, 0.02381877, 0.01140885, 0.01019156,
       0.01067219, 0.00886938, 0.0094812 , 0.00613977, 0.00361173,
       0.00184055, 0.00146633, 0.00113075, 0.00120265, 0.00216856,
       0.00749074, 0.0261295 , 0.06489394, 0.1190236 , 0.19054174,
       0.26422817, 0.34785515, 0.35106182, 0.29768446, 0.28392068,
       0.2479236 , 0.2729151 , 0.4002529 , 0.32621405, 0.22708407,
       0.18669172, 0.16173668, 0.17917942, 0.23462908, 0.2530375 ,
       0.20243469, 0.2221109 , 0.29609403, 0.21614796, 0.18348026,
       0.1607146 , 0.11130328, 0.05487794, 0.07270122, 0.10026675,
       0.06377813, 0.07610965, 0.11305903, 0.09689094, 0.05489435,
       0.06050633, 0.13506775, 0.06180866, 0.05573025, 0.03448793,
       0.08333092, 0.09102707, 0.11866996, 0.11876972, 0.10603561,
       0.04940936, 0.02720888, 0.04009135, 0.02461723, 0.02151141,
       0.0106258 , 0.00753845, 0.0174446 , 0.05965272, 0.19701

In [60]:
X_val.iloc[i]

Librosa_song     [-0.0684071, -0.09733205, -0.10176847, -0.0888...
Tonnetz          [-0.09441756, -0.09484137, -0.094202325, -0.09...
Chroma_stft_f    [0.45650896, 0.41325387, 0.48972058, 0.5651254...
Chroma_stft_s    [0.22323051, 0.26516527, 0.3266878, 0.33453786...
Name: 42, dtype: object

In [56]:
X_train_unified

In [57]:
X_val_unified

array([[-0.39563054, -0.39198002, -0.3778372 , ...,  0.15922691,
         0.20155372,  0.34691525],
       [-0.0684071 , -0.09733205, -0.10176847, ...,  0.5742422 ,
         0.53841484,  0.57365006],
       [ 0.39934954,  0.35114634,  0.36584747, ...,  0.4932235 ,
         0.53419447,  0.71227944],
       ...,
       [-0.02944762, -0.03251462, -0.03497472, ...,  0.3920835 ,
         0.4639663 ,  0.4111594 ],
       [-0.00946029, -0.00851459, -0.00759913, ...,  0.17840974,
         0.1526419 ,  0.11038169],
       [-0.09962035, -0.14447477, -0.15569057, ...,  0.00361607,
         0.00393412,  0.02213306]], dtype=float32)

In [40]:
X_val

,Librosa_song,Tonnetz,Chroma_stft_f,Chroma_stft_s,Tempogram_f,Tempogram_s
110,"[-0.017457193, -0.017624341, -0.015624179, -0....","[0.026353765, 0.024570197, 0.02275536, 0.02489...","[1.0, 0.5642115, 0.47739488, 0.47559193, 0.564...","[0.5733748, 0.42310804, 0.3925385, 0.34856766,...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.8043733293848835, 0.8046890185536487, 0.805..."
419,"[-0.03030353, -0.014233943, 0.0026929558, 0.01...","[-0.029143358, -0.024542235, -0.015208109, -0....","[0.6720495, 1.0, 1.0, 0.49888584, 0.24491909, ...","[0.70858324, 0.6802393, 0.66515374, 0.28105518...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.8295514182399509, 0.8300599662344182, 0.830..."
565,"[0.07826688, 0.08074787, 0.07867979, 0.0818728...","[0.06958537, 0.06772629, 0.06512518, 0.0672007...","[0.44145912, 0.6049853, 0.6421371, 0.63330364,...","[0.16214794, 0.3856984, 0.3925147, 0.4833511, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.668717699511237, 0.6693302411249694, 0.6699..."
77,"[0.6607101, 0.60325885, 0.5415306, 0.48279792,...","[0.5283116, 0.5084883, 0.46157172, 0.41345486,...","[0.8359728, 0.4047802, 0.2714141, 0.25420064, ...","[0.5063123, 0.42288557, 0.27528682, 0.14072967...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.8454512098790702, 0.845297410234466, 0.8451..."
181,"[-0.027863221, -0.025377138, -0.022002375, -0....","[-0.026653105, -0.025712471, -0.023295335, -0....","[1.0, 1.0, 0.62038106, 0.27252173, 0.21068275,...","[0.53195405, 0.7758484, 0.36931428, 0.22223967...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.9625062563556377, 0.9626234600495803, 0.962..."
...,...,...,...,...,...,...
117,"[-0.028132234, -0.026908375, -0.023900319, -0....","[-0.044372395, -0.042590223, -0.03883523, -0.0...","[0.19687948, 0.5495927, 0.36856797, 0.352479, ...","[0.1573559, 0.5790402, 0.45201787, 0.44815725,...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.7928159676118272, 0.7934168155100281, 0.794..."
382,"[-0.1085057, -0.06458457, -0.044254705, -0.022...","[-0.13412605, -0.1268544, -0.10958277, -0.0890...","[0.46285257, 0.39675078, 0.37262297, 0.9188177...","[0.307218, 0.3791471, 0.3209688, 0.4388499, 0....","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.9370614815262129, 0.9372832850545496, 0.937..."
405,"[0.39934954, 0.35114634, 0.36584747, 0.4353354...","[0.39187077, 0.39019248, 0.39938197, 0.4236725...","[0.2779876, 0.2800179, 0.40840977, 0.33449647,...","[0.35945225, 0.29428008, 0.49728826, 0.4643264...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.9074150554209395, 0.9073918925935048, 0.907..."
33,"[0.32638624, 0.26026922, 0.17868058, 0.1583147...","[0.24540785, 0.23271365, 0.20813797, 0.2014373...","[0.9557275, 0.99871296, 0.997962, 1.0, 0.58425...","[0.82788724, 0.86970335, 0.9350951, 0.9280264,...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.840908074469639, 0.8399449607541114, 0.8389..."


In [39]:
X_train

,Librosa_song,Tonnetz,Chroma_stft_f,Chroma_stft_s,Tempogram_f,Tempogram_s
593,"[0.02245393, 0.048207678, 0.060551748, 0.04588...","[0.032512184, 0.038693808, 0.045485076, 0.0399...","[0.12807852, 0.10184903, 0.18725187, 0.6678501...","[0.20089062, 0.113701545, 0.16216537, 0.419506...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.8090317620366206, 0.8092416802179471, 0.809..."
531,"[0.21809004, 0.24448387, 0.23326856, 0.2099215...","[0.22363892, 0.22111729, 0.2147647, 0.20480992...","[0.32349652, 0.5817766, 0.56464356, 0.70550007...","[0.30686578, 0.48902887, 0.46663973, 0.464649,...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.823203139992446, 0.8228947288381839, 0.8225..."
353,"[0.036012694, 0.01889443, -0.0008565266, -0.02...","[0.055436067, 0.049295645, 0.03385756, 0.01404...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.4528646, 0.3866359, 0.3681345, 0.36614066, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.8240470704936479, 0.8246981716846811, 0.825..."
332,"[0.03494117, 0.025747774, 0.017243618, 0.01300...","[0.015124373, 0.013248598, 0.00912195, 0.00664...","[0.12739271, 0.32744083, 0.39265493, 0.6606563...","[0.18719923, 0.32147688, 0.37015954, 0.5925821...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.7956254433365924, 0.7959389950867751, 0.796..."
534,"[0.01871043, 0.020348217, 0.022726415, 0.02632...","[0.020502754, 0.021342576, 0.024350075, 0.0271...","[0.34913695, 0.34909463, 0.2639558, 0.36960337...","[0.5952044, 0.5276708, 0.2751619, 0.27044314, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.6586490166032353, 0.6589996601878148, 0.659..."
...,...,...,...,...,...,...
71,"[-0.17614712, -0.16858773, -0.16201809, -0.155...","[-0.18967642, -0.18697233, -0.18079744, -0.174...","[1.0, 1.0, 1.0, 1.0, 1.0, 0.5137688, 1.0, 1.0,...","[0.838662, 0.86698085, 0.9159105, 0.87134075, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.5216926853323579, 0.5224635273316052, 0.523..."
106,"[0.07991847, 0.04860383, 0.021216983, -0.00151...","[0.06490308, 0.062255193, 0.055623535, 0.05058...","[0.93103355, 1.0, 1.0, 0.983292, 0.49883842, 0...","[0.5003422, 0.5610564, 0.5588547, 0.5825512, 0...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.7714983420178835, 0.7716118735218529, 0.771..."
270,"[0.18388945, 0.25324285, 0.25120065, 0.2020189...","[0.07810058, 0.0748787, 0.0687311, 0.059925936...","[0.29693505, 0.45254612, 0.5895548, 0.88208926...","[1.0, 1.0, 1.0, 1.0, 0.5157852, 0.15275894, 0....","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.8852057357244089, 0.8853720894302405, 0.885..."
435,"[-0.13953689, -0.1694999, -0.20757446, -0.2029...","[-0.09714573, -0.11987187, -0.1572257, -0.1696...","[0.85661477, 0.81266975, 0.7275551, 0.43242547...","[1.0, 0.83714736, 0.62288415, 0.45373365, 0.47...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.9373400904239327, 0.9374833546983692, 0.937..."


In [ ]:
X_train_unified

In [97]:
X_train_unified.shape, X_val_unified.shape, y_train_unified.shape, y_val_unified.shape

((148, 441862), (149, 441862), (148, 3), (149, 3))

In [96]:
set([len(e) for e in X_train_unified])

{441862}

In [ ]:
len(X_train)

In [ ]:
X_train.insert(loc=1 ,column='Unified_data', value=X_train_unified)
X_val["Unified_data"] = X_val_unified
y_train['Unified_result'] = y_train_unified
y_val['Unified_result'] = y_val_unified

In [ ]:
X_train

In [ ]:
X_train = X_train.drop(columns=['Librosa_song', 'Tonnetz', 'Chroma_stft_f', 'Chroma_stft_s', 'Tempogram_f', 'Tempogram_s'])
X_val = X_val.drop(columns=['Librosa_song', 'Tonnetz', 'Chroma_stft_f', 'Chroma_stft_s', 'Tempogram_f', 'Tempogram_s'])
y_train = y_train.drop(columns=['Genre_pop','Genre_rock'])
y_val = y_val.drop(columns=['Genre_pop','Genre_rock'])

In [ ]:
X_train

In [ ]:
X_train.shape[1:]

In [1]:
model = Sequential([
    Dense(512, activation="relu", input_shape = (len(X_train_unified), len(X_train_unified[0])) ),#X_train.values.shape[1:] ),
    Dense(10_000, activation="relu"),
    Dense(10_000, activation="relu"),
    Dense(10_000, activation="relu"),
    Dense(10_000, activation="relu"),
    Dense(10_000, activation="relu"),
    Dense(10_000, activation="relu"),
    Dense(10_000, activation="relu"),
    Dense(10_000, activation="relu"),
    Dense(len(y_train_unified[0]), activation="softmax")
])

NameError: name 'Sequential' is not defined

In [99]:
optimizer = optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer,
              loss="categorical_crossentropy",
              metrics=["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 148, 512)          226233856 
_________________________________________________________________
dense_1 (Dense)              (None, 148, 1024)         525312    
_________________________________________________________________
dense_2 (Dense)              (None, 148, 1024)         1049600   
_________________________________________________________________
dense_3 (Dense)              (None, 148, 1024)         1049600   
_________________________________________________________________
dense_4 (Dense)              (None, 148, 1024)         1049600   
_________________________________________________________________
dense_5 (Dense)              (None, 148, 1024)         1049600   
_________________________________________________________________
dense_6 (Dense)              (None, 148, 3)            3

In [ ]:
y_train

In [ ]:
for song in X_train_unified:
    for element in song:
        print(type(element))
        break
    break

In [ ]:
len(y_train_unified[0])

In [100]:
history = model.fit(np.asarray(X_train_unified), 
                    np.asarray(y_train_unified),
         validation_data=(np.asarray(X_val_unified), 
                          np.asarray(y_val_unified)),
         epochs=100,
         verbose=1,
         batch_size=32)

2021-10-20 20:06:35.556418: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-10-20 20:06:35.568702: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3800020000 Hz


Epoch 1/100
5/5 [==============================] - 5s 805ms/step - loss: 1.0715 - accuracy: 0.3566 - val_loss: 1.0743 - val_accuracy: 0.2886
Epoch 2/100
5/5 [==============================] - 3s 666ms/step - loss: 0.8743 - accuracy: 0.3622 - val_loss: 1.0181 - val_accuracy: 0.3154
Epoch 3/100
5/5 [==============================] - 3s 665ms/step - loss: 0.5872 - accuracy: 0.6687 - val_loss: 1.0076 - val_accuracy: 0.4295
Epoch 4/100
5/5 [==============================] - 3s 667ms/step - loss: 0.4511 - accuracy: 0.8143 - val_loss: 1.0199 - val_accuracy: 0.5570
Epoch 5/100
5/5 [==============================] - 3s 649ms/step - loss: 0.3997 - accuracy: 0.9921 - val_loss: 1.0211 - val_accuracy: 0.5973
Epoch 6/100
5/5 [==============================] - 3s 672ms/step - loss: 0.2169 - accuracy: 1.0000 - val_loss: 1.0972 - val_accuracy: 0.4698
Epoch 7/100
5/5 [==============================] - 3s 660ms/step - loss: 0.1143 - accuracy: 1.0000 - val_loss: 1.7340 - val_accuracy: 0.3624
Epoch 8/100
5

In [ ]:
history

In [227]:
a = np.array([1,2,3,4])
b = np.array([5,6,7,8])
c = np.concatenate((a,b),axis=0)
c.append(a)
c.append(b)
c = np.array(c)
c

AttributeError: 'numpy.ndarray' object has no attribute 'append'

In [269]:
def divied_song(song, sr, seconds=5):
    res = []
    for i in range(1, len(song)//(sr*seconds)):
        part = song[sr*seconds*(i-1):sr*seconds*i]
        res.append(part)
    return np.array(res)

In [273]:
def features(song, sr):
    res = []
    for part in song:
        chroma_stft = librosa.feature.chroma_stft(part, sr = sr)
        print(len(chroma_stft))
        aux = np.concatenate((part, chroma_stft),axis=None)
        harmonic = librosa.effects.harmonic(part)
        aux = np.concatenate((aux, harmonic),axis=None)
        res.append(aux)
    return np.array(res)

In [274]:
def song_to_predict(directory):
    x , sr = librosa.load(directory, mono=True, sr=44100)
    song = divied_song(x, sr, 5)
    feature = features(song, sr)
    return np.array(feature)

In [275]:
test = song_to_predict("test/blues.00020.wav")

12
12
12
12
12


In [247]:
test.shape

(5, 446172)

In [253]:
y_pred = model.predict(test)
y_pred

ValueError: in user code:

    /home/alvaro/miniconda3/envs/end/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:1478 predict_function  *
        return step_function(self, iterator)
    /home/alvaro/miniconda3/envs/end/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:1468 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/alvaro/miniconda3/envs/end/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/alvaro/miniconda3/envs/end/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/alvaro/miniconda3/envs/end/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/alva

In [ ]:
video_info = youtube_dl.YoutubeDL().extract_info(url = 'https://youtu.be/ujNeHIo7oTE' ,download=False)
video_info['title'] = clean_name(video_info['title'])
filename = f"music/test/{video_info['title']}.mp3"
options={
    'format':'bestaudio/best',
    'keepvideo':False,
    'outtmpl':filename,
}

with youtube_dl.YoutubeDL(options) as ydl:
    ydl.download([video_info['webpage_url']])

print("Download complete... {}".format(filename))

In [ ]:
music_album = {}
mono = True
songs_dir = 'test'
directories = [i for i in listdir("music/") if not i.startswith(".")]
music_album[songs_dir] = {}
for song in listdir(f"music/test"):
    add = True
    music_album[songs_dir] = []
    if not song.startswith(".") and not song == 'song_list':
        audio_data = f'music/{songs_dir}/{song}'
        x , sr = librosa.load(audio_data, mono=True, sr=44100)    #Cambiar a x , sr = librosa.load(audio_data, mono=False) para ver como afecta
        if mono:
            part_song = divide_by_sec(x, sr)
        else:
            if len(x) == 2:
                p1 = divide_by_sec(x[0], sr)
                p2 = divide_by_sec(x[1], sr)
                part_song = np.array(list(zip(p1,p2)))
            else:
                add = False
        if add:
            music_album[songs_dir].append(part_song)
music_album
lista_songs = []
for genero in music_album:
    for part_songs in music_album[genero][0]:
        lista_songs.append([part_songs, genero])

    dfsongs = pd.DataFrame(np.array(lista_songs), columns=['Librosa_song', 'Genre'])
    dfsongs = extract_features(dfsongs, mono)
    if mono:
        dfsongs = two_d_to_one(dfsongs)
dfsongs

In [ ]:
music_album

In [ ]:
dfsongs = dfsongs.drop(columns=["Genre"])
dfsongs

In [ ]:
test = unified_data(dfsongs)
test

In [ ]:
from tensorflow.keras.metrics import mean_squared_error as mse

In [ ]:
y_pred = model.predict(test)
np.mean(mse(y_pred,y_test))

## Correccion de los nombres de las canciones
A continuacion vamos a comprobar el nombre de las canciones que se van a descargar y vamos a introducir las partes del nombre que queremos borrar a la lista `to_clean` de la funcion `clean_name`. Asi, el nombre de las canciones será el correcto

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Sequential

In [8]:
def two_d_to_one(data):
    data['Chroma_stft_f'] = data['Chroma_stft'].apply(lambda x: x[0])
    data['Chroma_stft_s'] = data['Chroma_stft'].apply(lambda x: x[1])

    data['Flatness'] = data['Flatness'].apply(lambda x: x[0])

    data['Tempogram_f'] = data['Tempogram'].apply(lambda x: x[0])
    data['Tempogram_s'] = data['Tempogram'].apply(lambda x: x[1])

    data = data.drop(columns="Chroma_stft")
    data = data.drop(columns="Flatness")
    data = data.drop(columns="Tempogram")
    return data

def extract_features(data, mono):
    if mono:
        data['Chroma_stft'] = data['Librosa_song'].apply(lambda y : librosa.feature.chroma_stft(y = y, sr = 44100))
        data['Flatness'] = data['Librosa_song'].apply(lambda y :librosa.feature.spectral_flatness(y = y))
        data['Tonnetz'] = data['Librosa_song'].apply(lambda y :librosa.effects.harmonic(y = y))
        data['Tempogram'] = data['Librosa_song'].apply(lambda y:
            librosa.feature.tempogram(
                onset_envelope=librosa.onset.onset_strength(y = y, sr = 44100), 
                sr = 44100)
        )
        return data
    else:
        data['Chroma_stft_x'] = data['Librosa_song'].apply(lambda y : librosa.feature.chroma_stft(y = y[0], sr = 44100))
        data['Chroma_stft_y'] = data['Librosa_song'].apply(lambda y : librosa.feature.chroma_stft(y = y[1], sr = 44100))
        
        data['Flatness_x'] = data['Librosa_song'].apply(lambda y :librosa.feature.spectral_flatness(y = y[0]))
        data['Flatness_y'] = data['Librosa_song'].apply(lambda y :librosa.feature.spectral_flatness(y = y[1]))
        
        data['Tonnetz'] = data['Librosa_song'].apply(lambda y :np.array([librosa.effects.harmonic(y = y[0]),
                                                                           librosa.effects.harmonic(y = y[1])]
                                                                         ))
        data['Tempogram_x'] = data['Librosa_song'].apply(lambda y:
            librosa.feature.tempogram(
                onset_envelope=librosa.onset.onset_strength(y = y[0], sr = 44100), 
                sr = 44100)
        )
        data['Tempogram_y'] = data['Librosa_song'].apply(lambda y:
            librosa.feature.tempogram(
                onset_envelope=librosa.onset.onset_strength(y = y[1], sr = 44100), 
                sr = 44100)
        )
        return data
    
    
def create_music_album(mono):
    music_album = {}
    directories = [i for i in listdir("music/") if not i.startswith(".")]
    music_album['blues'] = {}
    list_part_song = []
    for song in listdir(f"music/blues"):
        add = True
        if not song.startswith(".") and not song == 'song_list':
            audio_data = f'music/blues/{song}'
            x , sr = librosa.load(audio_data, mono=mono, sr=44100)    #Cambiar a x , sr = librosa.load(audio_data, mono=False) para ver como afecta
            if mono:
                part_song = divide_by_sec(x, sr)
            else:
                if len(x) == 2:
                    p1 = divide_by_sec(x[0], sr)
                    p2 = divide_by_sec(x[1], sr)
                    part_song = np.array(list(zip(p1,p2)))
                else:
                    add = False
            if add:
                list_part_song.append(part_song)
    music_album["blues"]['Librosa_song'] = np.array(list_part_song)
    return music_album

def dataFrame_music(mono):
    music_album = create_music_album(mono)
    lista_songs = []
    return music_album
for song in music_album['blues']['Librosa_song']
    #for genero in music_album:
    #    for song in range(len(music_album[genero])):
    #        for part_songs in music_album[genero][song]:
    #            lista_songs.append([part_songs, genero])
            
    #dfsongs = pd.DataFrame(np.array(lista_songs), columns=['Librosa_song', 'Genre'])
    #dfsongs = extract_features(dfsongs, mono)
    #if mono:
    #    dfsongs = two_d_to_one(dfsongs)
    #return dfsongs

def divide_by_sec(audio, sr, seconds = 5):
    divided_audio = []
    for it in range(1, round(len(audio)/(seconds*sr)+1)):
        start = (it-1)*seconds*sr
        end = it*seconds*sr
        divided_audio.append(audio[start:end])
    return divided_audio

In [39]:
def features(music_album):
    
    for song_parts in range(len(music_album['blues']['Librosa_song'])):
        feature_parts = []
        chroma_stft = []
        for part in music_album['blues']['Librosa_song'][song_parts]:
            music_album['blues']['Chroma_stft'] = librosa.feature.chroma_stft(y = part, sr = 44100)
        print(len(music_album['blues']['Chroma_stft']))
        break

In [40]:
features(test)

12


In [9]:
test = dataFrame_music(True)

{'blues': {'Librosa_song': array([[[-2.40844376e-02, -3.23367454e-02, -2.99544483e-02, ...,
         -1.42604485e-01, -1.41752437e-01, -1.33600459e-01],
        [-1.18227124e-01, -9.87816155e-02, -7.86720812e-02, ...,
         -7.99948536e-03, -5.58138788e-02, -9.44879726e-02],
        [-1.15725666e-01, -1.19018942e-01, -1.10439882e-01, ...,
         -3.41648906e-02, -3.30818519e-02, -3.08240093e-02],
        [-2.84107216e-02, -2.64232699e-02, -2.48428155e-02, ...,
         -5.97077906e-02, -4.52578701e-02, -3.19154300e-02],
        [-1.96834691e-02, -8.30850005e-03,  2.50203372e-03, ...,
         -2.94735041e-02, -1.54403606e-02, -3.61622474e-03],
        [ 5.52220643e-03,  1.19466912e-02,  1.59010924e-02, ...,
          1.56222448e-01,  1.67575598e-01,  1.57748327e-01]],

       [[ 1.89388581e-02,  3.55602540e-02,  4.77707759e-02, ...,
          5.26043586e-02,  3.43628041e-02,  2.08565947e-02],
        [ 1.50757600e-02,  1.82716362e-02,  2.92662680e-02, ...,
          2.06649303e-01

In [15]:
test['blues']['Librosa_song'][0]

array([[-0.02408444, -0.03233675, -0.02995445, ..., -0.14260449,
        -0.14175244, -0.13360046],
       [-0.11822712, -0.09878162, -0.07867208, ..., -0.00799949,
        -0.05581388, -0.09448797],
       [-0.11572567, -0.11901894, -0.11043988, ..., -0.03416489,
        -0.03308185, -0.03082401],
       [-0.02841072, -0.02642327, -0.02484282, ..., -0.05970779,
        -0.04525787, -0.03191543],
       [-0.01968347, -0.0083085 ,  0.00250203, ..., -0.0294735 ,
        -0.01544036, -0.00361622],
       [ 0.00552221,  0.01194669,  0.01590109, ...,  0.15622245,
         0.1675756 ,  0.15774833]], dtype=float32)

In [ ]:
X_dummies = pd.get_dummies(test, columns=['Genre'])
y = X_dummies[re.findall(r"Genre\w*", " ".join(X_dummies.columns))]
X_dummies = X_dummies.drop(columns=re.findall(r"Genre\w*", " ".join(X_dummies.columns)))

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_dummies, y, random_state=42)

In [ ]:
def unified_data(dataFrame):
    index = []
    data = []
    for rows in dataFrame.iterrows():
        index.append(rows[0])
        unified_colums = []
        for column_data in rows[1].tolist():
            unified_colums.extend(column_data)
        data.append(unified_colums)
    return np.array(data)

def unified_result(dataFrame):
    index = []
    data = []
    for rows in dataFrame.iterrows():
        unified_colums = []
        for column_data in rows[1].tolist():
            unified_colums.append(np.array(column_data))
        data.append(np.array(unified_colums))
    return np.array(data)

In [ ]:
X_train_unified = unified_data(X_train)
X_val_unified = unified_data(X_val)
y_train_unified = unified_result(y_train)
y_val_unified = unified_result(y_val)

In [ ]:
X_train_unified.shape, X_val_unified.shape, y_train_unified.shape, y_val_unified.shape